<img style="float: right;width: 100px" src="https://www.enib.fr/images/logo-enib-accueil.jpg">

<div>
    <p><h3>ASN (S6)</h3></p>
    <p>Année 2020</p>
    <p><em>{{Nom}} {{Prénom}}</em></p>
</div>

<div style="text-align: center;padding-bottom:20px;padding-top:10px"><h1>Asservissements Numériques</h1>
    <h2>Labo 5 : Simulation d'une Régulation d'un alternateur</h2>
</div>    

In [40]:
import numpy as np
from control import tf, c2d, feedback, bode, pade, freqresp
from ENIB_control import step, impulse, nichols, rlocus, damp, pole, stepinfo, figure

## 1. But de l'étude

On se propose d’étudier la mise en œuvre de deux correcteurs numériques pour la régulation d’un processus analogique (un alternateur) caractérisé par sa fonction de transfert $F(p)$. Il s’agit de contrôler la valeur efficace de la tension alternative produite. On désire synthétiser le correcteur par une approche graphique (méthode fréquentielle dans le plan de Black). Le calcul de correcteurs et la validation des résultats se feront en utilisant au maximum les fonctionnalités de Python.

Ce TP comporte deux parties :
- une analyse du système à mettre en œuvre sous la forme d'une préparation théorique du système à effectuer avant le TP.
- une validation des résultats théoriques par simulation sous Jupyter Python, puis une synthèse de correcteurs dans le plan de Black-Nichols.


## 2. Description du Système

Le schéma de principe de l’alternateur est représenté sur la figure ci-dessous :


<figure style="padding:30px">
    <img src="./img/labo5_1.png" style="padding:20px; width: 600px" />
    <figcaption style="text-align:center">Fig.1 - Schéma de principe d’un alternateur.</figcaption>
</figure>

Le système est composé d’un stator (partie immobile) et d’un rotor (partie tournante). Le rotor est entraîné par un moteur annexe. Le fonctionnement d’un alternateur est exactement l’inverse d’un moteur électrique. Le rotor muni d’électro-aimants crée un champ magnétique tournant qui génère dans le stator (l’induit) un courant électrique. Une tension alternative de valeur efficace $U_s(t)$ apparaît alors aux bornes de la bobine. Il s’agit de la tension de sortie de l’alternateur qu’on souhaite réguler. La tension de sortie $U_s(t)$ dépend de l’aimantation dans l’inducteur (rotor), son contrôle s’effectue donc par le réglage du courant continu $i(t)$ fourni aux électro-aimants (si le courant croît, l’aimantation croît et la tension de sortie augmente). Le courant $i(t)$ est contrôlé par la tension continue $U_e(t)$ appliquée sur l’inducteur. L’alternateur peut être caractérisé par une fonction de transfert avec comme grandeur d’entrée, la tension continue $U_e(t)$ et comme grandeur de sortie, la valeur efficace de la tension en sortie notée $U_s(t)$.

La fonction de transfert $F_a(p)$ d’un alternateur peut être approchée par le modèle de Broïda : il s’agit d’une F.T. du 1er ordre de gain $A_0$ associée à un retard pur. 

$$F_a(p)=\frac{A_0}{1+Tp}e^{-\tau p}$$

La constante de temps est $T=0.5$s et le retard pur est de $\tau=150$ms. Le gain $A_0=5$ Veff/V (Veff :Volt efficace).


La tension à réguler, tension alternative à la sortie du processus en question, est mesurée au moyen d’un convertisseur AC/DC : une tension alternative (0 à 150 Veff maxi) est transformée en un signal continu proportionnel (0 à 10 V maxi). Ce "capteur" de tension élabore donc le signal de "retour" de la boucle (avec un simple gain $A_2$).

La fonction de régulation (réalisée par le correcteur) est effectuée un PC, muni d'un module d'entrée sortie CNA/CAN. Le convertisseur N-A est muni d'un B.O.Z. Le produit des gains $G_1$ et $G_2$ des convertisseurs est unitaire.

Le signal de commande en sortie du correcteur est fourni à l'alternateur via un amplificateur de puissance, de gain en tension $A_1 = 2$ V/V.

## 3. Préparation

L'ensemble du système peut être représenté par le schéma bloc de la figure 2.

<figure style="padding:30px">
  <img src="./img/labo5_2.png" style="padding:20px; width: 700px" />
   <figcaption style="text-align:center">Fig.2 - Schéma bloc du système.</figcaption>
</figure>


Ce système doit maintenant être représenté sous la forme classique d'un schéma synoptique avec retour unitaire comme sur la figure 3.

### 3.1 Fonction de transfert du système à asservir

* **Question** Déterminez la F.T. $F(p)$ de l’ensemble à asservir (amplificateur, processus analogique, convertisseur). Déterminez le gain statique $K$ de $F(p)$.

$$F(p)=\frac{A_0A_1A_2}{1+Tp}e^{-\tau p}$$

* avec $A_0=5$, $A_1=2$ et $A2=0.0666$.

* **Question** Donnez les caractéristiques (valeurs numériques) de la réponse indicielle de $F(p)$ : temps de réponse à $\pm 5\%$ de la valeur finale, dépassement relatif $D\%$, valeur finale.

* Gain statique : $F(0)=A_0A_1A_2=K=0.6666$
* valeur Finale : $0.6666$
* constante de temps : $T = 0.5$s
* Temps de réponse ($\pm 5\%$) : $1.5+0.15=1.65$s.
* Dépassement relatif : $0\%$

* **Question** Donnez l'expression du module (en dB) et de la phase (en degré) de $F(p)$ ($p=j\omega$). Calculez les valeurs du module et de la phase aux limites ($\omega=0$ et $\omega\to \infty$).

$$F(j\omega)=\frac{K}{1+j\omega T}e^{-j\tau \omega}$$

* Module : $|F(j\omega)|=\frac{|K|}{\sqrt{1+T^2\omega^2}}$
* Argument : $\arg[F(j\omega)]=-\arg[\omega T]-\tau\omega$

## 4. Analyse de la fonction de transfert $F(p)$ et $F(z)$.

Les calculs et les validations des résultats suivants se feront en utilisant au maximum les fonctionnalités de Python.

### 4.1 Saisie et Analyse de $F(p)$

La fonction de transfert du système est donnée par $F(p)=\frac{K}{1+0.5p}e^{-0.15p}$ où $K$ est déterminé
dans la préparation.

La fonction exponentielle, qui exprime le retard de $0.15$s, ne peut pas être écrite directement sous Python. Nous allons remplacer cette fonction exponentielle par l'approximation de Padé. 

<div class='alert alert-info'>
    En mathématiques, <a href="https://en.wikipedia.org/wiki/Padé_approximant">l'approximation de Padé</a> de la fonction exponentielle est une fraction
rationnelle $Delay(p)=\frac{N(p)}{D(p)}$ où $N(p)$ et $D(p)$ sont des polynômes de degré $k$ et $l$, telle que le  développement limité de la fraction à l'ordre $k + l$ soit identique à celui de l'exponentielle.
</div>
    
Nous utiliserons la fonction de Padé disponible sous python.control :

    [num, den]= pade(T,l)
    Delay=tf(num, den)

Les paramètres d'entrée sont le retard $T$ et le degré du dénominateur $l$ de la fonction de Padé. La fonction `pade` doit être importée de la librairie `control` (`from control import pade`). Les paramètres de sortie sont les coefficients des polynômes de Padé.


* **Question** Déterminez en fonction de $C(z)$ et $F(z)$ (pas d'applications numériques) 

   * l'expression de la fonction de transfert en boucle fermée : $H(z)$,
   * l'expression du rapport $err(z)$ où $\epsilon(z)$ désigne signal d'erreur,
   * l'expression du rapport $cmd(z)$ où $U_c(z)$ désigne le signal de commande.

\begin{align}
H(z)&=\frac{C(z)F(z)}{1+C(z)F(z)}\\
err(z)&=\frac{E(z)-S(z)}{E(z)}=1-H(z)\\
cmd(z)&=\frac{U_c(z)}{S(z)}\times \frac{S(z)}{E(z)}=\frac{H(z)}{F(z)} 
\end{align}


* **Question**
    * Saisissez la FT $F(p)$, notée `f` sous forme polynomiale, sans le retard :`f0 = tf([K], [0.5, 1]`
    * Créer la variable `Delay` pour $\tau =0.15$ s en prenant $j=1$. Saisissez alors la fonction $F(p)$ : `f = f0*Delay`
    * Tracez la réponse indicielle et vérifiez que vous retrouvez les caractéristiques théoriques obtenues dans la préparation.

**remarque** : Vous pouvez tester diverses valeurs du degré `j` de la fonction de Padé et vérifier que
l'approximation est d'autant meilleure que`j` est grand.


In [60]:
K = 0.6666
f0 = tf([K],[0.5,1])
[num, den]= pade(0.15,1)
Delay=tf(num, den)
f = f0*Delay

print(2/3*0.95)
fig = figure("time")
fig.plot(f)
fig.show()

0.6333333333333333


Par la suite on gardera une fonction de Padé d'ordre $1$ (`j=1`)`


* **Question**
    * Tracez les réponses fréquentielles des fonctions de transfert avec et sans retard f0 et f dans le plan de Black-Nichols. Observez le déphasage produit par le retard pur .
    * Complétez le tableau suivant pour F(p) pour des pulsations comprises entre 0,01 et 20 rad/s.
    * Vérifiez vos résultats en comparant aux résultats obtenus dans la préparation.


<table class="table">
<thead>
<tr>
    <th>$\omega$ (rad/s)</th>
    <th>0.01</th>
    <th>0.1</th>
    <th>1</th>
    <th>3</th>
    <th>6</th>
    <th>9</th>
    <th>12</th>
    <th>15</th>
    <th>20</th>
    <th>$\infty$</th>
</tr>
</thead>
<tbody>
<tr>
    <th>Gain (dB)</th>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
</tr>
<tr>
    <th>Phase (deg)</th>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
    <td>...</td>
</tr>
</tboby>
</table>

In [61]:
mag,phase,w = freqresp(f,omega=[0.01,0.1,1,3,6,9,12,15,20,1000])
print("mag: {}".format(20*np.log10(mag)))
print("phase: {}".format(180*phase/np.pi))

fig = figure("bode")
fig.plot(f)
fig.show()

mag: [[[ -3.52280239  -3.53353763  -4.49179394  -8.64152742 -13.52269381
   -16.79628316 -19.20471105 -21.10044872 -23.56590755 -57.50211127]]]
phase: [[[  -0.37242016   -3.7218258   -35.14335783  -81.67069946 -120.02054181
   -145.50989227 -164.51210278 -179.13827796  163.09072819   91.64238833]]]


### 4-2 Analyse de $F(z)$

On se propose d'échantillonner à $T_e = 0.05 s$. 

* **Question**
    * Déterminez la fonction de transfert en B.O. $F(z)$ avec BOZ.
    * Tracez la réponse indicielle et vérifiez que vous retrouvez les caractéristiques théoriques obtenues dans la préparation.
    * Tracez la réponse fréquentielle de $F(z)$ dans le plan de Black-Nichols et comparez là avec la réponse fréquentielle de $F(p)$.

In [62]:
Te = 0.05
Fz = c2d(f,Te)
print(Fz)


 -0.02865 z + 0.05952
----------------------
z^2 - 1.418 z + 0.4646

dt = 0.05



* **Question** Tracez la réponse indicielle et vérifiez que vous retrouvez les caractéristiques théoriques obtenues dans la préparation.
    

In [63]:
fig = figure("time")
fig.plot(Fz)
fig.show()

* **Question** Tracez la réponse fréquentielle de $F(z)$ dans le plan de Black-Nichols et comparez là avec la réponse fréquentielle de $F(p)$.

In [66]:
fig = figure("nichols")
fig.plot(f)
fig.plot(Fz)
fig.grid(show_phase=False)
fig.show()

## 5. Régulation proportionnelle.

### 5.1 Calibration du correcteur

On veut obtenir une réponse indicielle, pour le système corrigé et bouclé, analogue à celle d’un système du 2ème ordre de facteur d'amortissement $m = 0.38$ (donc, pour la réponse harmonique, un facteur de résonance de $M =3$ dB).
La période d'échantillonnage est fixée à $Te = 0.05$ s. Le correcteur proportionnel s'écrit sous la forme d'un simple gain : $C(z) = K$.

Pour déterminer la valeur à donner au gain $K$ du régulateur, utilisez la méthode graphique :

* **Question** 
    * tracez la FT de la chaîne directe du système $KF(z)$ et cherchez à obtenir la résonance voulue sur la fonction de transfert en boucle fermée $H(z)$ par retouches successives de la valeur de K. Donnez la valeur du gain $K$ ainsi obtenu
    * Donnez le gain statique de $H(z)$ en valeur naturelle.
    * Déterminez à partir de $\omega_r$, et à l'aide des abaques que vous avez à votre disposition le temps de réponse prévisible de $H(z)$.
    * Déterminez la marge de gain et la marge de phase du système corrigé.

In [70]:
K = 3.5
fig = figure("nichols")
fig.plot(K*Fz)
fig.grid(show_phase=False)
fig.show()

Nous pouvons prendre $K=3.5$. Nous obtenons alors un gain statique en BF de $-3dB$ (0.7).

### 5.2 Validation du correcteur proportionnel.

* **Question** 
    * Calculez avec python, $H(z)$, la fonction de transfert du système bouclé et corrigé avec le gain $K$.
    * Tracez la réponse indicielle de $H(z)$. Déterminez les caractéristiques de cette réponse ($tr$, $D\%$, $m$, $s[\infty]$).

 
  
 
 

In [71]:
Hz = feedback(K*Fz,1)

fig = figure("time")
fig.plot(Hz)
fig.show()

## 6. Correcteur proportionnel intégral

On désire corriger le système par un correcteur PI afin d'obtenir un comportement, en boucle fermée, analogue à un système discret Hm(z) du second ordre caractérisé par un facteur de résonance autour de 3 dB (amortissement $m = 0.38$) et une erreur de position nulle. La période d'échantillonnage est fixée à $Te = 0.05$ s.

La synthèse de ce correcteur PI est réalisée par une méthode de synthèse classique dans le plan de Black-Nichols. On se propose ici d'implémenter un correcteur proportionnel intégral numérique.


### 6.1 Calibration du correcteur

* **Question** 
    * Déterminez graphiquement dans le plan de Black-Nichols le gain $K$ et les constantes d'intégration $Ti$. Prendre $Ti >5/\omega_r$ (Utilisez la méthodologie décrite dans la labo 4)
    * Déduisez-en l'équivalent numérique du correcteur analogique PI proposé, en utilisant l'approximation rectangulaire par excès de l'intégration:
    
$$C(z) = K\left(1+\frac{T_e}{T_i}\frac{z}{z-1}\right)$$

In [76]:
K = 4.5
K1 = 0.8
wr = 8.2
Ti = 5/wr
Cz = K+K*(Te/Ti)*tf([1,0],[1,-1])
Cz2 = K1*Cz
fig = figure("nichols")
fig.plot(K*Fz)
fig.plot(Cz*Fz)
fig.plot(Cz2*Fz)
fig.grid(show_phase=False)
fig.show()

### 6.2 Validation du correcteur proportionnel.

* **Question** 
    * Calculez avec python, $H(z)$, la fonction de transfert du système bouclé et corrigé.
    * Tracez la réponse indicielle de $H(z)$. Déterminez les caractéristiques de cette réponse ($tr$, $D\%$, $m$, $s[\infty]$).



In [82]:
sys_list = [K*Fz,Cz*Fz,Cz2*Fz]

fig = figure("time")
for sys in sys_list:
    H = feedback(sys,1)
    fig.plot(H)

fig.show()